In [1]:
# Import required libraries
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import torch
import open_clip
from sklearn.decomposition import PCA
import warnings

# Configuration
DATA_PATH = Path('data')

# Set preferences
plt.style.use('default')
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
# Load all required data
print("Loading data...")

# Load CLIP embeddings and metadata
clip_file = DATA_PATH / 'CLIP_ViT-H-14_embeddings_structured.npz'
with np.load(clip_file) as data:
    clip_embeddings = data['embeddings'][:50]  # (50, 72, 1024)
    object_nums = data['object_nums'][:50]     # sorted object IDs
    angles = data['angles']               # sorted rotation angles

# Load original images
images_file = DATA_PATH / 'coil100_images_compressed.npz'
with np.load(images_file) as data:
    images = data['images'].astype(np.uint8)[:50]  # (50, 72, H, W, C)

# Load PCA projections for both spaces
input_pca_file = DATA_PATH / 'coil100_pca.npz'
clip_pca_file = DATA_PATH / 'CLIP_ViT-H-14_pca.npz'
input_pca = np.load(input_pca_file)['pca_projections'].astype(np.float32)[:50]  # (50, 72, 2)
clip_pca = np.load(clip_pca_file)['pca_projections'].astype(np.float32)[:50]    # (50, 72, 2)

# Load precomputed analysis matrices
distances_file = DATA_PATH / 'clip_pca_mean_distances.npz'
subspace_file = DATA_PATH / 'clip_pca_subspace_angles.npz'
clip_mean_dist_matrix = np.load(distances_file)['mean_dist_matrix'][:50, :50]
subspace_angle_matrix = np.load(subspace_file)['subspace_angle_matrix'][:50, :50]  # (50, 50, 2)

# Create visualization helpers
cmap = plt.get_cmap('twilight')
angle_colors = cmap(np.linspace(0, 1, len(angles)))
input_pca_dict = {int(object_nums[i]): input_pca[i] for i in range(len(object_nums))}
clip_pca_dict = {int(object_nums[i]): clip_pca[i] for i in range(len(object_nums))}

print(f"Loaded {len(object_nums)} objects with {len(angles)} angles each")

Loading data...
Loaded 50 objects with 72 angles each


## 🎯 Interactive Visualizations Overview

Welcome! In this notebook, you'll find a series of interactive visualizations designed to help you explore how objects are represented and transformed in both image and model spaces. We'll start simple and build up to more complex ideas, so you can get an intuitive feel for the data and what the model is learning.

## 1. Input Space PCA Visualization: How Do Object Images Change as They Rotate?

Let's begin by looking at the raw images themselves, before any neural network gets involved. This first interactive plot lets you see how the appearance of an object changes as it rotates, and how those changes look when we reduce the image data to just two dimensions using PCA (Principal Component Analysis).

**How to use this plot:**
- Use the dropdown menu to pick any object (from 1 to 50).
- Use the slider to choose a rotation angle for that object.

**What you'll see:**
- On the left: the actual image of the selected object at your chosen angle.
- On the right: a scatter plot showing all 72 images of that object (one for each angle), each as a point in 2D PCA space. The currently selected angle is highlighted.

**What to notice:**
- For most objects, the points will form a circle or loop. This means that as the object rotates, its pixel values change smoothly and regularly—a pattern that PCA captures nicely.
- Try picking different objects and angles. Do all objects form nice circles? Are there exceptions?

**Why this matters:** If rotation already creates a clear, simple structure in the raw pixel space, it gives us a baseline for what we might expect (or not expect) to see in deeper model layers.

In [3]:
# Widget selectors for input space visualization
object_selector = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[0]),
    description='Object:',
    continuous_update=False
)
angle_slider = widgets.IntSlider(
    min=int(angles[0]),
    max=int(angles[-1]),
    step=5,
    value=int(angles[0]),
    description='Angle:',
    continuous_update=False
)

def show_image_and_pca(obj_num, angle):
    """Display an object image and its PCA projection across all angles."""
    try:
        i = np.where(object_nums == obj_num)[0][0]
        j = np.where(angles == angle)[0][0]
        img = images[i, j]
        projs = input_pca_dict[obj_num]  # shape: (72, 2)

        fig, axs = plt.subplots(1, 2, figsize=(10, 4))

        # Show image
        axs[0].imshow(img)
        axs[0].axis('off')
        axs[0].set_title(f'Object {obj_num}, Angle {angle}°')

        # Show PCA projection
        axs[1].scatter(projs[:, 0], projs[:, 1], c=angle_colors, alpha=0.5, s=40)
        axs[1].scatter(projs[j, 0], projs[j, 1], color=angle_colors[j], s=120,
                       edgecolor='black', alpha=1, label='Selected angle')
        axs[1].set_xlabel('PCA 1')
        axs[1].set_ylabel('PCA 2')
        axs[1].set_title('Input Space PCA Projections')
        axs[1].axis('equal')
        axs[1].legend()

        plt.tight_layout()
        plt.show()

    except IndexError:
        plt.figure(figsize=(3, 3))
        plt.text(0.5, 0.5, 'Data not found', ha='center', va='center')
        plt.axis('off')
        plt.show()

# Interactive display
ui = widgets.HBox([object_selector, angle_slider])
out = widgets.interactive_output(show_image_and_pca, {'obj_num': object_selector, 'angle': angle_slider})
display(ui, out)


Output()

## 2. Comparing Visual and CLIP Representations: How Does CLIP "See" Rotations?

In this section, you can explore how CLIP interprets object rotations compared to the original images. The interactive plot below lets you pick any object and rotation angle to see how its appearance and representation change.

**How to use this plot:**  
- Use the dropdown to select an object.  
- Adjust the angle slider to rotate the object.  
- Watch how the three panels update:
     - **Left:** The actual image of the object at your chosen angle.
     - **Middle:** The object's rotation pattern in pixel (input) space, visualized with PCA.
     - **Right:** The same rotation pattern, but as CLIP "sees" it—showing how CLIP transforms visual information into its own semantic space.

**What to look for:**  
- Notice how the CLIP space (right panel) often reshapes the rotation pattern. Sometimes it stays circular, but often it bends into arcs, horseshoes, or lines.  
- Try different objects—smooth, textured, simple, or complex—and see how their patterns change. This reveals how CLIP balances raw visual features with its learned, semantic understanding of objects.

Take a moment to play with the controls and see which objects keep their shape in CLIP space, and which ones get transformed!

In [4]:
# Widget selectors for comparison
object_selector_comp = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[0]),
    description='Object:',
    continuous_update=False
)
angle_slider_comp = widgets.IntSlider(
    min=int(angles[0]),
    max=int(angles[-1]),
    step=5,
    value=int(angles[0]),
    description='Angle:',
    continuous_update=False
)

def show_combined_pca(obj_num, angle):
    """Compare object representation in input space vs CLIP space."""
    try:
        i = np.where(object_nums == obj_num)[0][0]
        j = np.where(angles == angle)[0][0]

        img = images[i, j]
        input_proj = input_pca_dict[obj_num]
        clip_proj = clip_pca_dict[obj_num]

        fig, axs = plt.subplots(1, 3, figsize=(15, 4))

        # Original image
        axs[0].imshow(img)
        axs[0].axis('off')
        axs[0].set_title(f'Object {obj_num}, Angle {angle}°')

        # Input space PCA
        axs[1].scatter(input_proj[:, 0], input_proj[:, 1], c=angle_colors, alpha=0.5, s=40)
        axs[1].scatter(input_proj[j, 0], input_proj[j, 1], color=angle_colors[j], s=120,
                       edgecolor='black', label='Selected angle')
        axs[1].set_title('Input Space PCA')
        axs[1].set_xlabel('PCA 1')
        axs[1].set_ylabel('PCA 2')
        axs[1].axis('equal')
        axs[1].legend()

        # CLIP space PCA
        axs[2].scatter(clip_proj[:, 0], clip_proj[:, 1], c=angle_colors, alpha=0.5, s=40)
        axs[2].scatter(clip_proj[j, 0], clip_proj[j, 1], color=angle_colors[j], s=120,
                       edgecolor='black', label='Selected angle')
        axs[2].set_title('CLIP Space PCA')
        axs[2].set_xlabel('PCA 1')
        axs[2].set_ylabel('PCA 2')
        axs[2].axis('equal')
        axs[2].legend()

        plt.tight_layout()
        plt.show()

    except IndexError:
        plt.figure(figsize=(3, 3))
        plt.text(0.5, 0.5, 'Data not found', ha='center', va='center')
        plt.axis('off')
        plt.show()

# Interactive display
ui = widgets.HBox([object_selector_comp, angle_slider_comp])
out = widgets.interactive_output(show_combined_pca, {'obj_num': object_selector_comp, 'angle': angle_slider_comp})
display(ui, out)


Output()

## 3. Comparing How Objects Encode Rotation: Subspace Angles

In this section, we explore how different objects represent rotation in CLIP space by comparing the orientation of their 2D rotational subspaces. This helps us understand whether objects that are close together in CLIP space also "encode" rotation in similar ways.

**What does the interactive plot show?**  
The interactive plot below lets you select two objects and a viewing angle. For each object, you'll see:
- The image of the object at the chosen angle.
- A visualization of its 2D rotation subspace in CLIP space.

The plot also computes the *maximum principal angle* between the two objects' rotation subspaces. This angle tells us how well-aligned the two rotation planes are:  
- **Smaller angles** mean the objects encode rotation in a more similar way.
- **Larger angles** mean their rotation subspaces are more different.

**How to use the plot:**  
- Use the dropdown menus to pick two objects (Object X and Object Y) to compare.
- Use the angle slider to choose a specific rotation angle for both objects.
- The plot will update to show the selected images, their subspaces, and the alignment score.

**Background:**  
To compare the subspaces, we use principal angles computed via SVD. For two objects' subspace bases $Q_A$ and $Q_B$, the principal angles $\theta_1, \theta_2$ are given by:
$$
M = Q_A^\top Q_B \in \mathbb{R}^{2\times 2},\quad M = U\Sigma V^\top,\quad \Sigma = \mathrm{diag}(\sigma_1, \sigma_2)
$$
where $\sigma_i = \cos\theta_i$. The **maximum principal angle** $\Theta_{\max} = \max(\theta_1, \theta_2)$ is used as a conservative measure of alignment.

**Try it out:**  
See if objects that are visually or semantically similar also have better-aligned rotation subspaces!

In [5]:
# Widget selectors for subspace angle comparison
object_selector_x = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[0]),
    description='Object X:',
    continuous_update=False
)
object_selector_y = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[1]),
    description='Object Y:',
    continuous_update=False
)
angle_slider_sub = widgets.IntSlider(
    min=int(angles[0]),
    max=int(angles[-1]),
    step=5,
    value=int(angles[0]),
    description='Angle:',
    continuous_update=False
)

def show_subspace_comparison(obj_num_x, obj_num_y, angle):
    """Compare two objects and their subspace angle in CLIP space."""
    try:
        ix = np.where(object_nums == obj_num_x)[0][0]
        iy = np.where(object_nums == obj_num_y)[0][0]
        j = np.where(angles == angle)[0][0]
        
        img_x = images[ix, j]
        img_y = images[iy, j]
        clip_pca_x = clip_pca_dict[obj_num_x]
        clip_pca_y = clip_pca_dict[obj_num_y]

        # Get maximum subspace angle
        max_angle_deg = np.max(subspace_angle_matrix[ix, iy])

        fig, axs = plt.subplots(1, 4, figsize=(18, 4))
        
        # Object X image
        axs[0].imshow(img_x)
        axs[0].axis('off')
        axs[0].set_title(f'Object {obj_num_x}')
        
        # Object X PCA
        axs[1].scatter(clip_pca_x[:, 0], clip_pca_x[:, 1], c=angle_colors, alpha=0.5, s=40)
        axs[1].scatter(clip_pca_x[j, 0], clip_pca_x[j, 1], color=angle_colors[j], s=120,
                       edgecolor='black', alpha=1, label=f'Angle {angle}°')
        axs[1].set_xlabel('PCA 1')
        axs[1].set_ylabel('PCA 2')
        axs[1].set_title(f'Object {obj_num_x} CLIP PCA')
        axs[1].axis('equal')
        axs[1].legend()
        
        # Object Y PCA
        axs[2].scatter(clip_pca_y[:, 0], clip_pca_y[:, 1], c=angle_colors, alpha=0.5, s=40)
        axs[2].scatter(clip_pca_y[j, 0], clip_pca_y[j, 1], color=angle_colors[j], s=120,
                       edgecolor='black', alpha=1, label=f'Angle {angle}°')
        axs[2].set_xlabel('PCA 1')
        axs[2].set_ylabel('PCA 2')
        axs[2].set_title(f'Object {obj_num_y} CLIP PCA')
        axs[2].axis('equal')
        axs[2].legend()
        
        # Object Y image
        axs[3].imshow(img_y)
        axs[3].axis('off')
        axs[3].set_title(f'Object {obj_num_y}')
        
        plt.suptitle(f'Maximum Subspace Angle: {max_angle_deg:.2f}°', fontsize=14)
        plt.tight_layout()
        plt.show()

    except IndexError:
        plt.figure(figsize=(3, 3))
        plt.text(0.5, 0.5, 'Data not found', ha='center', va='center')
        plt.axis('off')
        plt.show()

# Interactive display
ui = widgets.HBox([object_selector_x, angle_slider_sub, object_selector_y])
out = widgets.interactive_output(
    show_subspace_comparison, 
    {'obj_num_x': object_selector_x, 'obj_num_y': object_selector_y, 'angle': angle_slider_sub}
)
display(ui, out)

Output()

## 4. Subspace Angle vs Distance: Exploring Relationships in CLIP Space

Let's explore how similar or different objects are in CLIP's representation space, and whether that relates to how their "rotation subspaces" are oriented. In other words: do objects that are close together in CLIP space also rotate in similar ways?

**What you'll see below:**  
- Each object is represented by a set of CLIP embeddings, one for each rotation angle.
- For each object, we compute the average embedding ($\mu_i$) and the main plane of rotation in CLIP space (using PCA).
- For every pair of objects, we calculate:
    - The distance between their average embeddings ($d_{ij}$).
    - The maximum angle between their rotation planes ($\Theta_{ij}$).

**How to use the interactive plot:**  
- Use the dropdowns to select two objects.
- The plot will show their distance in CLIP space and the angle between their rotation subspaces.
- Try different pairs to see if objects that are close together also have more similarly oriented rotation subspaces.

This helps us test the hypothesis: *Objects that are close in CLIP space tend to have more similarly oriented rotational subspaces.*

Mathematically:
- For object $i$ with embeddings $\{f(x_i^\theta)\}$:
  $$
  \mu_i = \frac{1}{|\Theta|}\sum_{\theta} f(x_i^\theta),\qquad U_i\in\mathbb{R}^{d\times 2}\ \text{(PCA basis of its rotation plane)}.
  $$
- For a pair $(i,j)$:
  $$
  d_{ij}=\lVert \mu_i - \mu_j\rVert_2,\qquad
  \Theta_{ij}=\max\big(\arccos(\sigma_1),\arccos(\sigma_2)\big)\ \text{from } U_i^\top U_j.
  $$

In [6]:
# Compute pairwise statistics for scatter plot
n = len(object_nums)
pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
distances = []
angles_max = []

for i, j in pairs:
    distances.append(clip_mean_dist_matrix[i, j])
    angles_max.append(np.max(subspace_angle_matrix[i, j]))
    
distances = np.array(distances)
angles_max = np.array(angles_max)

In [7]:
# Widget selectors for distance analysis
object_selector_dist_x = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[0]),
    description='Object X:',
    continuous_update=False
)
object_selector_dist_y = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[1]),
    description='Object Y:',
    continuous_update=False
)

# Output widgets for images
img_out_x = widgets.Output()
img_out_y = widgets.Output()

def update_images(obj_num_x, obj_num_y):
    """Update the displayed images for selected objects."""
    img_out_x.clear_output(wait=True)
    img_out_y.clear_output(wait=True)
    
    with img_out_x:
        try:
            ix = np.where(object_nums == obj_num_x)[0][0]
            img_x = images[ix, 0]
            plt.figure(figsize=(3, 3))
            plt.imshow(img_x)
            plt.axis('off')
            plt.title(f'Object {obj_num_x}')
            plt.show()
        except Exception:
            plt.figure(figsize=(3, 3))
            plt.text(0.5, 0.5, 'Image not found', ha='center', va='center')
            plt.axis('off')
            plt.show()
            
    with img_out_y:
        try:
            iy = np.where(object_nums == obj_num_y)[0][0]
            img_y = images[iy, 0]
            plt.figure(figsize=(3, 3))
            plt.imshow(img_y)
            plt.axis('off')
            plt.title(f'Object {obj_num_y}')
            plt.show()
        except Exception:
            plt.figure(figsize=(3, 3))
            plt.text(0.5, 0.5, 'Image not found', ha='center', va='center')
            plt.axis('off')
            plt.show()

def plot_distribution(obj_num_x, obj_num_y):
    """Plot distance vs angle distribution with selected pair highlighted."""
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot all pairs
    ax.scatter(distances, angles_max, alpha=0.3, s=30, label='All pairs')
    
    # Highlight selected pair
    try:
        ix = np.where(object_nums == obj_num_x)[0][0]
        iy = np.where(object_nums == obj_num_y)[0][0]
        if ix < iy:
            sel_idx = pairs.index((ix, iy))
        else:
            sel_idx = pairs.index((iy, ix))
        ax.scatter([distances[sel_idx]], [angles_max[sel_idx]], 
                  color='red', s=150, label='Selected pair', 
                  edgecolor='black', linewidth=2, zorder=5)
    except Exception:
        pass
    
    ax.set_xlabel('Mean Euclidean Distance in CLIP PCA Space', fontsize=12)
    ax.set_ylabel('Maximum Subspace Angle (degrees)', fontsize=12)
    ax.set_title('Relationship Between Distance and Subspace Angle', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Set up observers
object_selector_dist_x.observe(lambda c: update_images(object_selector_dist_x.value, object_selector_dist_y.value), names='value')
object_selector_dist_y.observe(lambda c: update_images(object_selector_dist_x.value, object_selector_dist_y.value), names='value')

# Initial image display
update_images(object_selector_dist_x.value, object_selector_dist_y.value)

# Layout
vbox_x = widgets.VBox([object_selector_dist_x, img_out_x])
vbox_y = widgets.VBox([object_selector_dist_y, img_out_y])
top_row = widgets.HBox([vbox_x, vbox_y])

# Interactive plot
out = widgets.interactive_output(
    plot_distribution, 
    {'obj_num_x': object_selector_dist_x, 'obj_num_y': object_selector_dist_y}
)

display(top_row, out)

Output()

## 5. Exploring CLIP: Maximizing Text-Image Similarity
Here, you can see how CLIP connects images and text by mapping both into a shared space and measuring how well they match. The goal is to experiment with different text descriptions for a given image and see which ones CLIP thinks are the best match.

**How to use this interactive plot:**
- Select an object from the dropdown menu to view its image.
- In the text box, type a description you think matches the image.
- Click "Compute Similarity" to see how closely your text matches the image according to CLIP.
- Try different descriptions to see what features or words make the similarity score go up or down.

This tool helps you understand what kinds of details or phrases CLIP pays attention to when matching images and text. It's a fun way to probe the model's "thought process" and see how it interprets both visual and textual information.

**A bit of context:**  
CLIP works by comparing L2-normalized image and text embeddings using a scaled dot product (often cosine similarity). During training, it learns to bring matching image-text pairs closer together and push mismatched pairs apart. Here, we're interested in the similarity score between your text and the selected image—the higher the score, the better the match!


In [8]:
# Initialize CLIP model
print("Loading CLIP model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms(
    "ViT-H-14", pretrained="laion2b_s32b_b79k"
)
tokenizer = open_clip.get_tokenizer('ViT-H-14')
model = model.float().eval().to(device)
print(f"Model loaded on {device}")

Loading CLIP model...
Model loaded on cpu


In [9]:
# Create widgets
object_selector_sim = widgets.Dropdown(
    options=[int(o) for o in object_nums],
    value=int(object_nums[0]),
    description='Object:',
)
text_input = widgets.Text(
    value='',
    placeholder='Describe what you see...',
    description='Text:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)
compute_button = widgets.Button(
    description='Compute Similarity',
    button_style='primary',
    icon='check'
)

# Output areas
image_output = widgets.Output()
results_output = widgets.Output()

# History storage for each object
similarity_history = {}

def show_selected_image(obj_num):
    """Display the selected object image."""
    image_output.clear_output(wait=True)
    with image_output:
        ix = np.where(object_nums == obj_num)[0][0]
        img = images[ix, 0]  # Use first angle
        plt.figure(figsize=(4, 4))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Object {obj_num}')
        plt.show()

def compute_similarity():
    """Compute CLIP similarity between image and text."""
    obj_num = object_selector_sim.value
    description = text_input.value.strip()
    
    if not description:
        return
    
    # Get image
    ix = np.where(object_nums == obj_num)[0][0]
    img = images[ix, 0]
    pil_img = Image.fromarray(img)
    
    # Compute embeddings
    with torch.no_grad():
        # Image embedding
        image_input = preprocess(pil_img).unsqueeze(0).to(device)
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        
        # Text embedding
        text_input_tensor = tokenizer([description]).to(device)
        text_features = model.encode_text(text_input_tensor)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        # Compute similarity
        similarity = (image_features @ text_features.T).item()
    
    # Store in history
    if obj_num not in similarity_history:
        similarity_history[obj_num] = []
    similarity_history[obj_num].append((description, similarity))
    
    # Clear input
    text_input.value = ''
    
    # Update display
    update_results_display()

def update_results_display():
    """Update the results display with history and plot."""
    results_output.clear_output(wait=True)
    with results_output:
        obj_num = object_selector_sim.value
        if obj_num in similarity_history and similarity_history[obj_num]:
            history = similarity_history[obj_num]
            
            # Show last 5 attempts
            print("Recent attempts:")
            print("-" * 50)
            for i, (desc, sim) in enumerate(history[-5:], 1):
                print(f"{i}. \"{desc}\"")
                print(f"   Similarity: {sim:.4f}")
                print()
            
            # Find best attempt
            best_desc, best_sim = max(history, key=lambda x: x[1])
            print(f"Best attempt: \"{best_desc}\"")
            print(f"Best similarity: {best_sim:.4f}")
            
            # Plot progress
            if len(history) > 1:
                plt.figure(figsize=(8, 4))
                attempts = range(1, len(history) + 1)
                similarities = [sim for _, sim in history]
                
                plt.plot(attempts, similarities, 'b-o', markersize=8, linewidth=2)
                plt.xlabel('Attempt Number', fontsize=12)
                plt.ylabel('Cosine Similarity', fontsize=12)
                plt.title(f'Similarity Progress for Object {obj_num}', fontsize=14)
                plt.grid(True, alpha=0.3)
                plt.ylim(-0.1, 1.1)
                
                # Highlight best attempt
                best_idx = np.argmax(similarities)
                plt.plot(best_idx + 1, similarities[best_idx], 'r*', markersize=20)
                
                plt.tight_layout()
                plt.show()

def on_object_change(change):
    """Handle object selection change."""
    show_selected_image(object_selector_sim.value)
    update_results_display()

def on_button_click(b):
    """Handle compute button click."""
    compute_similarity()

def on_text_submit(change):
    """Handle text input submission (Enter key)."""
    if not change['new'] and not change['old']:
        # Ignore initial trigger
        return
    if change['name'] == 'value' and change['type'] == 'change':
        compute_similarity()

# Connect event handlers
object_selector_sim.observe(on_object_change, names='value')
compute_button.on_click(on_button_click)
# Set .continuous_update to False and observe value changes for text_input
text_input.continuous_update = False
text_input.observe(on_text_submit, names='value')

# Initial display
show_selected_image(object_selector_sim.value)

# Create layout
input_row = widgets.HBox([object_selector_sim, text_input, compute_button])
display(input_row)
display(image_output)
display(results_output)

Output()

Output()